# FEAD-D: Facial Expression Analysis in Deepfake Detection
---
Michela Gravina, Antonio Galli, Geremia De Micco, Stefano Marrone, Giuseppe Fiameni, Carlo Sansone




Before running the code, please:

*  Change the runtime setting by selecting the GPU as the accelerator.
*  (Optional) Connect the notebook to your Drive
*  Download the weights of the models at this link: https://drive.google.com/drive/folders/1jeoqIl6SH5eRZooVesM2gqtBuTxOJbiU?usp=sharing


## Import libraries



In [ ]:
!pip install pickle5

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import zipfile
import cv2
from tensorflow import keras
import tensorflow as tf
import keras
import math
from keras.models import load_model,model_from_json
from keras.models import Sequential
from keras.applications.inception_v3 import InceptionV3
from random import randint
from random import sample
import random
import dlib
import pickle5 as pickle
from keras.layers import Dense, Dropout, TimeDistributed
from tensorflow.keras.layers import LSTM, GRU, Bidirectional
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import time

print(sys.version)
np.version.version
import tensorflow as tf
print(tf.__version__)
print(keras.__version__)

In [ ]:
!nvidia-smi

## Function Definition

The implemented framework consists of the following modules:

* **Face detection**, in which the target video is analyzed to detect one or more faces.
* **Features Extraction**, that analyzes the texture of target images and extracts the emotion from the detected face frame by frame.
* **Features temporal analysis**, in which all the features extracted in the previous stages are analyzed together in a cross-frame fashion to spot incoherent and unnatural patterns in the emotional evolution of the target subject.


### Step 1: Face detection

In [ ]:
def increseContrast(frame):
    lab= cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)

    limg = cv2.merge((cl,a,b))
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    final_frame = cv2.hconcat((frame, final)) # or vconcat for vertical concatenation

    return final


def increaseBrightnessBrg(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

def automaticBrightnessContrastBrg(image, clip_hist_percent=25):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Calculate grayscale histogram
    hist = cv2.calcHist([gray],[0],None,[256],[0,256])
    hist_size = len(hist)
    # Calculate cumulative distribution from the histogram
    accumulator = []
    accumulator.append(float(hist[0]))
    for index in range(1, hist_size):
        accumulator.append(accumulator[index -1] + float(hist[index]))
    # Locate points to clip
    maximum = accumulator[-1]
    clip_hist_percent *= (maximum/100.0)
    clip_hist_percent /= 2.0
    # Locate left cut
    minimum_gray = 0
    while accumulator[minimum_gray] < clip_hist_percent:
        minimum_gray += 1
    # Locate right cut
    maximum_gray = hist_size -1
    while accumulator[maximum_gray] >= (maximum - clip_hist_percent):
        maximum_gray -= 1
    # Calculate alpha and beta values
    alpha = 1
    try:
        alpha = 255 / (maximum_gray - minimum_gray)
    except ZeroDivisionError:
        pass

    beta = -minimum_gray * alpha

    auto_result = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    splitImgs = cv2.hconcat((frame, auto_result)) # or vconcat for vertical concatenation
    return auto_result


# Crop
def crop(img, x, y, w, h, margin):
    x -= margin
    y -= margin
    w += margin * 2
    h += margin * 2
    if x < 0:
        x = 0
    if y <= 0:
        y = 0
    return img[y:y + h, x:x + w]

def sqareMarginBBOX(top, bottom, left, right, fromInterpolation):
    global frame_FSHS
    global frameToCut
    if fromInterpolation == True:
        h, w, c = frameToCut.shape
    else:
        h, w, c = frame_FSHS.shape
    minLeft = 0
    minTop = 0
    maxBottom = h
    maxRight = w
    maxDim = max((bottom - top), (right - left))
    margin = maxDim * 0.5

    newTop = int(top - margin)
    offSetTop = 0
    if newTop < 0:
        offSetTop = math.ceil(abs(newTop))
        newTop = 0

    newBottom = int(top + maxDim + margin + offSetTop)
    if newBottom > maxBottom:
        offSetBottom = math.ceil(abs(newBottom - maxBottom))
        newBottom = maxBottom
        newTop = newTop - offSetBottom

    newLeft = int(left - margin)
    offSetLeft = 0
    if newLeft < 0:
        offSetLeft = math.ceil(abs(newLeft))
        newLeft = 0

    newRight = int(left + maxDim + margin + offSetLeft)
    if newRight > maxRight:
        offSetRight = math.ceil(abs(newRight - maxRight))
        newRight = maxRight
        newLeft = newLeft - offSetRight
    box = int(newTop), int(newRight), int(newBottom), int(newLeft)
    return box

def cropBBox(box, fromInterpolation):
    newTop, newRight, newBottom, newLeft = box
    newTop = int(newTop)
    newRight = int(newRight)
    newBottom = int(newBottom)
    newLeft = int(newLeft)
    if newTop < 0:
        newTop = 0
        newRight = int(newRight)
        newBottom = int(newBottom)
        newLeft = int(newLeft)
    if newLeft < 0:
        newLeft = 0
    if abs(newBottom - newTop) != squareDim_noZoom:
        newBottom = newTop + squareDim_noZoom
    if abs(newRight - newLeft) != squareDim_noZoom:
        newRight = newLeft + squareDim_noZoom

    if abs(newBottom - newTop) != abs(newRight - newLeft):
        z = 0
    if fromInterpolation == True:
        faceInFrame = frameToCut[newTop:newBottom,newLeft:newRight]  # crop frame_FSHS ************************************************
    else:
        faceInFrame = frame_FSHS[newTop:newBottom, newLeft:newRight]  # crop frame_FSHS ************************************************
    return faceInFrame

def resizeAndAppend():
    global facesDetected
    global frames_48
    global frames_299
    global frames_380_B3_convLSTM
    global frame_id
    global last_detected_face_48
    global last_detected_face_299
    global back_interp_frame_index
    global middle_interp_frame_index
    global foreward_interp_frame_index
    global count
    global index_detected
    global bbox_last_detected_faces
    global first_frame
    global last_BBox
    global squareDim_noZoom
    global first_frame_BBox

    facesDetected = facesDetected + 1

    moreConfidenceFace = 0
    max_conf_score = 0
    max_distance_from_choosedFace = 1000
    if len(detected_faces_raw)>1:
        for i in range(len(detected_faces_raw)):
            if first_frame:
                if detected_faces_raw[i].confidence > max_conf_score:
                    max_conf_score = detected_faces_raw[i].confidence
                    moreConfidenceFace = i
            else:
                (t1, r1, b1, l1) = first_frame_BBox
                l2 = detected_faces_raw[i].rect.left()
                t2 = detected_faces_raw[i].rect.top()
                r2 = detected_faces_raw[i].rect.right()
                b2 = detected_faces_raw[i].rect.bottom()
                (xc1, yc1) = (l1 + r1 / 2, t1 + b1 / 2)
                (xc2, yc2) = (l2 + r2 / 2, t2 + b2 / 2)

                if abs(xc1-xc2) < max_distance_from_choosedFace:
                    max_distance_from_choosedFace = abs(xc1-xc2)
                    moreConfidenceFace = i


    #top, right, bottom, left = detected_faces_raw[0]
    left = detected_faces_raw[moreConfidenceFace].rect.left()
    top = detected_faces_raw[moreConfidenceFace].rect.top()
    right = detected_faces_raw[moreConfidenceFace].rect.right()
    bottom = detected_faces_raw[moreConfidenceFace].rect.bottom()
    if left<0:
        left=0
    if top<=0:
        top=0
    #box = top, right, bottom, left
    index_detected.append(count)

    curret_sqaured_box = sqareMarginBBOX(top, bottom, left, right, False)

    if first_frame:
        first_frame = False
        last_BBox = curret_sqaured_box
        top, right, bottom, left = curret_sqaured_box
        squareDim_noZoom = max(abs(bottom - top),abs(right - left))
        first_frame_BBox = curret_sqaured_box

    else:
        (t1, r1, b1, l1) = last_BBox
        (t2, r2, b2, l2) = curret_sqaured_box
        dim_1 = max(abs(b1-t1),abs(r1-l1))
        dim_2 = max(abs(b2-t2),abs(r2-l2))
        (xc1, yc1) = (l1 + r1 / 2, t1 + b1 / 2)
        (xc2, yc2) = (l2 + r2 / 2, t2 + b2 / 2)

        dx_up = False
        dx_down = False
        sx_up = False
        sx_down = False
        if xc2 >= xc1:
            if yc2 >= yc1:
                dx_up = True
            else:
                dx_down = True
        else:
            if yc2 >= yc1:
                sx_up = True
            else:
                sx_down = True
        alpha = 0.1
        delta_x = float(abs(xc2 - xc1))*alpha
        delta_y = float(abs(yc2 - yc1))*alpha
        if dx_up == True:
            newbbox = (float(t1) + delta_y , float(l1) + delta_x + squareDim_noZoom , float(t1) + delta_y + squareDim_noZoom, float(l1) + delta_x)
        else:
            if dx_down == True:
                newbbox = (float(t1) - delta_y , float(l1) + delta_x  + squareDim_noZoom , float(t1) - delta_y + squareDim_noZoom, float(l1) + delta_x)
            else:
                if sx_up == True:
                    newbbox = (float(t1) + delta_y , float(l1) - delta_x  + squareDim_noZoom, float(t1) + delta_y + squareDim_noZoom, float(l1) - delta_x)
                else:
                    if sx_down == True:
                        newbbox = (float(t1) - delta_y , float(l1) - delta_x  + squareDim_noZoom, float(t1) - delta_y + squareDim_noZoom, float(l1) - delta_x)
        last_BBox = newbbox


    bbox_last_detected_faces.append(last_BBox)
    faceInFrame = cropBBox(last_BBox, False)

    frames_380_B3_convLSTM.append(faceInFrame)

    frame_48_gray = cv2.cvtColor(faceInFrame, cv2.COLOR_BGR2GRAY)
    frame_299_rgb = cv2.cvtColor(faceInFrame, cv2.COLOR_BGR2RGB)

    resizedFrame_48 = cv2.resize(frame_48_gray, (48, 48),
                                interpolation=cv2.INTER_LINEAR)
    resizedFrame_299 = cv2.resize(frame_299_rgb, (299, 299),
                                interpolation=cv2.INTER_LINEAR)

    last_detected_face_48 = resizedFrame_48
    last_detected_face_299 = resizedFrame_299

    faceRGB = cv2.cvtColor(faceInFrame, cv2.COLOR_BGR2RGB)
    face_380_rgb = cv2.resize(faceRGB, (380, 380), interpolation=cv2.INTER_LINEAR)
    frames_48.append(resizedFrame_48)
    frames_299.append(resizedFrame_299)
    frame_id.append(count)


# Interpolation methods
def checkInterpolationAndResize():
    resizeAndAppend()
    global frames_48
    global frames_299
    global frame_id
    global facesInterpolated
    global count
    global back_interp_frame_index
    global middle_interp_frame_index
    global index_detected
    global frames_back_interp
    global frames_middle_interp
    global frames_forward_interp

    if len(back_interp_frame_index) > 0:
        (t1, r1, b1, l1) = bbox_last_detected_faces[len(bbox_last_detected_faces) - 1]
        (t2, r2, b2, l2) = bbox_last_detected_faces[len(bbox_last_detected_faces) - 2]

        dim_1 = max(abs(b1-t1),abs(r1-l1))
        dim_2 = max(abs(b2-t2),abs(r2-l2))

        (xc1, yc1) = (l1 + r1 / 2, t1 + b1 / 2)
        (xc2, yc2) = (l2 + r2 / 2, t2 + b2 / 2)
        dx_up = False
        dx_down = False
        sx_up = False
        sx_down = False
        if xc2 >= xc1:
            if yc2 >= yc1:
                dx_up = True
            else:
                dx_down = True
        else:
            if yc2 >= yc1:
                sx_up = True
            else:
                sx_down = True
        index_diff = index_detected[len(index_detected) - 1] - index_detected[len(index_detected) - 2]
        delta_x = 1
        try:
            delta_x = float(abs(xc2 - xc1)) / float(index_diff)
        except ZeroDivisionError:
            pass

        delta_y = 1
        try:
            delta_y = float(abs(yc2 - yc1)) / float(index_diff)
        except ZeroDivisionError:
            pass

        num_box_to_create = len(back_interp_frame_index)
        for i in range(num_box_to_create):
            if dx_up == True:
                temp_top = float(t1) - delta_y *(i + 1)
                temp_left = float(l1) - delta_x * (i + 1)
                temp_bottom = temp_top + squareDim_noZoom
                temp_right = temp_left + squareDim_noZoom
                newbbox = (temp_top,temp_right,temp_bottom,temp_left)
                resizeAppendIinterpolatedFrames(newbbox, True, False, False, i)
            else:
                if dx_down == True:
                    temp_top = float(t1) + delta_y *i
                    temp_left = float(l1) - delta_x * i
                    temp_bottom = temp_top + squareDim_noZoom
                    temp_right = temp_left + squareDim_noZoom
                    newbbox = (temp_top,temp_right,temp_bottom,temp_left)
                    resizeAppendIinterpolatedFrames(newbbox, True, False, False, i)
                else:
                    if sx_up == True:
                        temp_top = float(t1) - delta_y *i
                        temp_left = float(l1) + delta_x * i
                        temp_bottom = temp_top + squareDim_noZoom
                        temp_right = temp_left + squareDim_noZoom
                        newbbox = (temp_top,temp_right,temp_bottom,temp_left)
                        resizeAppendIinterpolatedFrames(newbbox, True, False, False, i)
                    else:
                        if sx_down == True:
                            temp_top = float(t1) + delta_y *i
                            temp_left = float(l1) + delta_x * i
                            temp_bottom = temp_top + squareDim_noZoom
                            temp_right = temp_left + squareDim_noZoom
                            newbbox = (temp_top,temp_right,temp_bottom,temp_left)
                            resizeAppendIinterpolatedFrames(newbbox, True, False, False, i)
        frames_back_interp = []
        back_interp_frame_index = []

    if len(middle_interp_frame_index) > 0:
        (t1, r1, b1, l1) = bbox_last_detected_faces[len(bbox_last_detected_faces) - 1]
        (t2, r2, b2, l2) = bbox_last_detected_faces[len(bbox_last_detected_faces) - 2]
        dim_1 = max(abs(b1-t1),abs(r1-l1))
        dim_2 = max(abs(b2-t2),abs(r2-l2))

        (xc1, yc1) = (l1 + r1 / 2, t1 + b1 / 2)
        (xc2, yc2) = (l2 + r2 / 2, t2 + b2 / 2)
        dx_up = False
        dx_down = False
        sx_up = False
        sx_down = False
        if xc2 >= xc1:
            if yc2 >= yc1:
                dx_up = True
            else:
                dx_down = True
        else:
            if yc2 >= yc1:
                sx_up = True
            else:
                sx_down = True
        num_box_to_create = len(middle_interp_frame_index)
        delta_x = float(abs(xc2 - xc1)) / float(num_box_to_create + 1)
        delta_y = float(abs(yc2 - yc1)) / float(num_box_to_create + 1)
        for i in range(num_box_to_create):
            if dx_up == True:
                newbbox = (float(t1) + delta_y , float(l1) + delta_x + squareDim_noZoom , float(t1) + delta_y + squareDim_noZoom, float(l1) + delta_x)  # sommo o sottraggo il delta a seconda del movimento
                resizeAppendIinterpolatedFrames(newbbox, False, True, False, i)
            else:
                if dx_down == True:
                    newbbox = (float(t1) - delta_y , float(l1) + delta_x  + squareDim_noZoom , float(t1) - delta_y + squareDim_noZoom, float(l1) + delta_x)
                    resizeAppendIinterpolatedFrames(newbbox, False, True, False, i)
                else:
                    if sx_up == True:
                        newbbox = (float(t1) + delta_y , float(l1) - delta_x  + squareDim_noZoom, float(t1) + delta_y + squareDim_noZoom, float(l1) - delta_x)
                        resizeAppendIinterpolatedFrames(newbbox, False, True, False, i)
                    else:
                        if sx_down == True:
                            newbbox = (float(t1) - delta_y , float(l1) - delta_x  + squareDim_noZoom, float(t1) - delta_y + squareDim_noZoom, float(l1) - delta_x)
                            resizeAppendIinterpolatedFrames(newbbox, False, True, False, i)
        frames_middle_interp = []
        middle_interp_frame_index = []

def forwardInterpolation():
    global forward_interp_frame_index
    global frames_forward_interp
    forward_interp_frame_index = middle_interp_frame_index
    frames_forward_interp = frames_middle_interp

    (t1, r1, b1, l1) = bbox_last_detected_faces[len(bbox_last_detected_faces) - 1]
    (t2, r2, b2, l2) = bbox_last_detected_faces[len(bbox_last_detected_faces) - 2]

    dim_1 = max(abs(b1-t1),abs(r1-l1))
    dim_2 = max(abs(b2-t2),abs(r2-l2))

    (xc1, yc1) = (l1 + r1 / 2, t1 + b1 / 2)
    (xc2, yc2) = (l2 + r2 / 2, t2 + b2 / 2)

    dx_up = False
    dx_down = False
    sx_up = False
    sx_down = False
    if xc2 >= xc1:

        if yc2 >= yc1:
            dx_up = True
        else:
            dx_down = True
    else:
        if yc2 >= yc1:
            sx_up = True
        else:
            sx_down = True

    index_diff = index_detected[len(index_detected) - 1] - index_detected[len(index_detected) - 2]
    delta_x = 1
    try:
        delta_x = float(abs(xc2 - xc1)) / float(index_diff)
    except ZeroDivisionError:
        pass

    delta_y = 1
    try:
        delta_y = float(abs(yc2 - yc1)) / float(index_diff)
    except ZeroDivisionError:
        pass

    num_box_to_create = len(forward_interp_frame_index)

    for i in range(num_box_to_create):
        if dx_up == True:
                newbbox = (float(t2) + delta_y , float(l2) + delta_x + squareDim_noZoom , float(t2) + delta_y + squareDim_noZoom, float(l2) + delta_x)
                resizeAppendIinterpolatedFrames(newbbox, False, True, False, i)
        else:
            if dx_down == True:
                newbbox = (float(t2) - delta_y , float(l2) + delta_x  + squareDim_noZoom , float(t2) - delta_y + squareDim_noZoom, float(l2) + delta_x)
                resizeAppendIinterpolatedFrames(newbbox, False, True, False, i)
            else:
                if sx_up == True:
                    newbbox = (float(t2) + delta_y , float(l2) - delta_x  + squareDim_noZoom, float(t2) + delta_y + squareDim_noZoom, float(l2) - delta_x)
                    resizeAppendIinterpolatedFrames(newbbox, False, True, False, i)
                else:
                    if sx_down == True:
                        newbbox = (float(t2) - delta_y , float(l2) - delta_x  + squareDim_noZoom, float(t2) - delta_y + squareDim_noZoom, float(l2) - delta_x)
                        resizeAppendIinterpolatedFrames(newbbox, False, True, False, i)
    frames_forward_interp = []
    forward_interp_frame_index = []

def resizeAppendIinterpolatedFrames(newbbox, back, middle, forw, index):
    global frames_middle_interp
    global frames_48
    global frames_299
    global frame_id
    global facesInterpolated
    global frameToCut

    top, right, bottom, left = newbbox
    if middle == True:
        frameToCut = frames_middle_interp[index]
    else:
        if back == True:
            frameToCut = frames_back_interp[index]
        else:
            if forw == True:
                frameToCut = frames_forward_interp[index]

    temp_bbox = int(top), int(right), int(bottom), int(left),
    face_interp = cropBBox(temp_bbox, True)
    if face_interp.size == 0:
        print('empty list')
    else:
        frames_380_B3_convLSTM.append(face_interp)

        frame_48_gray = cv2.cvtColor(face_interp, cv2.COLOR_BGR2GRAY)
        frame_299_rgb = cv2.cvtColor(face_interp, cv2.COLOR_BGR2RGB)
        resizedFrame_48 = cv2.resize(frame_48_gray, (48, 48),
                                    interpolation=cv2.INTER_LINEAR)
        resizedFrame_299 = cv2.resize(frame_299_rgb, (299, 299),
                                    interpolation=cv2.INTER_LINEAR)
        faceRGB = cv2.cvtColor(face_interp, cv2.COLOR_BGR2RGB)
        face_380_rgb = cv2.resize(faceRGB, (380, 380), interpolation=cv2.INTER_LINEAR)
        frames_48.append(resizedFrame_48)
        frames_299.append(resizedFrame_299)


        if middle == True:
            frame_id.append(middle_interp_frame_index[index])
        else:
            if back == True:
                frame_id.append(back_interp_frame_index[index])
            else:
                if forw == True:
                    frame_id.append(forward_interp_frame_index[index])
        facesInterpolated = facesInterpolated + 1


# Utils

def createVideoFile(frame_rate, size, channels, array,videoPathName):
    fourcc_mp4 = cv2.VideoWriter_fourcc(*'MP4V') #cv2.VideoWriter_fourcc(*'DIVX')
    size = size
    videoOut = cv2.VideoWriter(f'{videoPathName}.mp4',fourcc_mp4, frame_rate, size, channels)
    for i in range(len(array)):
        img = array[i].astype('uint8')
        videoOut.write(img)
        videoOut.release()



def name_frame_num_Video(vid):
    cap = cv2.VideoCapture(vid)
    frame_num = cap.get(7)
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    nameVideo = vid.rsplit("/")[-1]
    print(f'video {nameVideo}, frame_num {frame_num}, fps {fps}')
    return nameVideo, frame_num, fps

### Step2: Features Extraction

In [ ]:
def features_extraction_step2(x_data_48, x_data_299, fer67_cnn, incV3_cnn ):
  x_data_48 = np.expand_dims(x_data_48, axis=-1)
  predictions2048_48 = fer67_cnn.predict(x_data_48, batch_size = 16)

  columns_Fer67_2048 =[('f_Fer' + str(i)) for i in range(2048)] #f1,f2,f3....fn
  df_fer67_2048Features = pd.DataFrame(predictions2048_48,columns = columns_Fer67_2048)
  df_fer67_2048Features['frame_id'] = np.arange(df_fer67_2048Features.shape[0]).tolist()
  df_fer67_2048Features = df_fer67_2048Features.set_index('frame_id')

  x_data_preproc_299 = keras.applications.inception_v3.preprocess_input(x_data_299)
  predictions_299 = incV3_cnn.predict(x_data_preproc_299, batch_size = 16)
  columns_InceptionV3 =[('f_Inc' + str(i)) for i in range(2048)] #f1,f2,f3....fn
  df_incV3_2048Features = pd.DataFrame(predictions_299,columns = columns_InceptionV3)
  df_incV3_2048Features['frame_id'] = np.arange(df_incV3_2048Features.shape[0]).tolist()
  df_incV3_2048Features = df_incV3_2048Features.set_index('frame_id')
  merged_df_fer67_IncV3 = pd.merge(df_fer67_2048Features, df_incV3_2048Features, left_index=True, right_index=True, how='inner',validate="one_to_one")
  return merged_df_fer67_IncV3

def importModels(dataMainDirectory):
    print('LOADING MODELS...')
    print('------------------------------------ LOADING fer67AccMicroExpreDetector')
    json_file = open(f'{dataMainDirectory}fer67AccMicroExpreDetector.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(f'{dataMainDirectory}fer67AccMicroExpreDetector.h5')
    from keras.models import Sequential
    fer67_cnn = Sequential()
    for layer in loaded_model.layers[:-7]: # just exclude last layer from copying
        fer67_cnn.add(layer)

    print('------------------------------------ LOADING InceptV3')

    incV3_cnn = InceptionV3(include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling='max',
    )

    return fer67_cnn, incV3_cnn

### Step 3: Features temporal analysis

In [ ]:
def classification_model_fc(base_path):
  time_steps = 300
  features = 4096
  model = Sequential()
  model.add(Bidirectional(LSTM(units = 1024,return_sequences=True)))
  model.add(Bidirectional(LSTM(units = 512, return_sequences=True)))
  model.add(Bidirectional(LSTM(units = 512, return_sequences=False)))
  model.add(Dense(512, activation = 'relu'))
  model.add(Dense(256, activation = 'relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.build((None, time_steps, features ))
  model.summary()
  model.load_weights(base_path + 'model_weights.h5')
  return model

## Prediction



**dataMainDirectory** is the path to the folder containing the model weights:


*   fer67AccMicroExpreDetector.h5 / fer67AccMicroExpreDetector.json
*   mmod_human_face_detector.dat
*   model_weights.h5

**path_name** is the path to the folder containing the videos to classify

Please update the dataMainDirectory and path_name variables according to the location where you intend to upload the necessary files.



In [ ]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    print(f'device: {device}')
    tf.config.experimental.set_memory_growth(device, True)


dataMainDirectory = '/content/drive/MyDrive/ICIAP2023_SUBMISSION/'  #base path
path_name = dataMainDirectory + '/' + 'VideoFolder'   #path to video folder
testSet_folder_path = path_name

fer67_cnn, incV3_cnn = importModels(dataMainDirectory)
classification_model = classification_model_fc(dataMainDirectory)
all_video_files = os.listdir(testSet_folder_path)
testSet_name_list = [name_frame_num_Video(str(testSet_folder_path + '/' + vid)) for vid in all_video_files]
print(testSet_name_list)
testSet_df = pd.DataFrame(testSet_name_list, columns = ['filename','n_of_frames','fps'])
testSet_df.set_index('filename', inplace=True)
# METADATA
testSet = testSet_df
testSet.sort_index(inplace=True)
min_N = min(testSet['n_of_frames'])
print('-----------------------------------------------')

#import face_recognition
print(f'IS DLIB USING CUDA? {dlib.DLIB_USE_CUDA}')
#first N frames
N = 300                 # changed next
numeroDiVideo = len(testSet.index)
print(numeroDiVideo)
videoNonCaricati = 0
i = 0
video_start = 0
test_video_path = path_name
last_video_to_detect = 0
tot_prediction= pd.DataFrame()

face_detector_cnn = dlib.cnn_face_detection_model_v1(dataMainDirectory + '/' + 'mmod_human_face_detector.dat')
log_df = pd.DataFrame(columns=['video_name','n_of_orginal_frames','detected_faces','n_interpolation','sample','frame_rate'])


print(f'Video from {video_start} to {numeroDiVideo}')

for nameVideo in all_video_files[video_start:numeroDiVideo]:
  try:
    since = time.time()
    frame_rate = testSet.loc[nameVideo]['fps']
    ast_video_to_detect = last_video_to_detect + 1
    #gc.collect()
    path_video_file = f'{test_video_path}/{nameVideo}'
    print(f'\n Reading: {path_video_file}')
    frames_48 = []
    frames_299 = []
    frames_380_B3_convLSTM = []
    ofVideo = []
    frame_id = []
    facesDetected = 0
    facesInterpolated = 0
    facesAugmented = 0
    count = 0
    first_frame = True    #TRACKING

    # INTERPOLATION properties
    bbox_last_detected_faces = []
    index_detected = []
    back_interp_frame_index = []
    frames_back_interp = []
    middle_interp_frame_index = []
    frames_middle_interp = []
    foreward_interp_frame_index = []
    frames_forward_interp = []
    frame_i = False
    last_detected_frame_index = 0


    N_OF_FRAMES = int(testSet.loc[nameVideo]['n_of_frames'])
    N = N_OF_FRAMES
    cap = cv2.VideoCapture(path_video_file)
    while count < N:
      resizedFrame = []
      success, frame = cap.read()
      count = count + 1
      if success==True:
        ofVideo.append(nameVideo)
        frame_FSHS = increseContrast(frame)
        grayContrast = cv2.cvtColor(frame_FSHS, cv2.COLOR_BGR2GRAY)
        detected_faces_raw = face_detector_cnn(grayContrast, 1)

        if len(detected_faces_raw) > 0:
          last_detected_frame_index = count
          if frame_i == False:
            frame_i = True
            resizeAndAppend()
          else:
            checkInterpolationAndResize()
        else:
          brightPlus_frame = increaseBrightnessBrg(frame)
          moreContrastFrame = increseContrast(brightPlus_frame)
          grayContrast = cv2.cvtColor(moreContrastFrame, cv2.COLOR_BGR2GRAY)
          detected_faces_raw = face_detector_cnn(grayContrast, 1)
          if len(detected_faces_raw) > 0:
              last_detected_frame_index = count
              if frame_i == False:
                  frame_i = True
                  resizeAndAppend()
              else:
                  checkInterpolationAndResize()
          else:
              autom_brigh_contr_frame = automaticBrightnessContrastBrg(frame)
              detected_faces_raw = face_detector_cnn(autom_brigh_contr_frame, 1)
              if len(detected_faces_raw) > 0:
                  last_detected_frame_index = count
                  if frame_i == False:
                      frame_i = True
                      resizeAndAppend()
                  else:
                      checkInterpolationAndResize()
              else:
                  autom_brigh_contr_frame = automaticBrightnessContrastBrg(frame)
                  grayContrast_brigh = cv2.cvtColor(autom_brigh_contr_frame, cv2.COLOR_BGR2GRAY)
                  detected_faces_raw = face_detector_cnn(grayContrast_brigh, 1)
                  if len(detected_faces_raw) > 0:
                      last_detected_frame_index = count
                      if frame_i == False:
                          frame_i = True
                          resizeAndAppend()
                      else:
                          checkInterpolationAndResize()
                  else:

                      if facesDetected == 0:
                          back_interp_frame_index.append(count)
                          frames_back_interp.append(frame)
                      else:
                          middle_interp_frame_index.append(count)
                          frames_middle_interp.append(frame)

    if last_detected_frame_index == N or last_detected_frame_index == N_OF_FRAMES:
      a = 1
    else:
      if not last_detected_frame_index == 0:
        forwardInterpolation()
      else:
        print('error during cv2.read...')
    cap.release()

    ofVideo = [nameVideo] * (facesDetected + facesInterpolated)
    all_frames_id = []
    last_frame_id_list = frame_id
    all_frames_id.extend(last_frame_id_list)
    data_48 = np.array(frames_48)
    reversed_data_48 = data_48[::-1]
    data_299 = np.array(frames_299)
    reversed_data_299 = data_299[::-1]

    if not last_detected_frame_index == 0:
      all300_frames_48 = data_48
      all300_frames_299 = data_299
      i = i + 1

      #-------------------------------------------------------------------------------------------------- STEP 2
      print('FEATURES EXTRACTION')
      print(all300_frames_48.shape)
      print(all300_frames_299.shape)
      merged_df_fer67_IncV3 = features_extraction_step2(all300_frames_48, all300_frames_299, fer67_cnn, incV3_cnn )

      #-------------------------------------------------------------------------------------------------- STEP 3
      print('CLASSIFICATION')
      n_rows = 300

      x = [ element for element in merged_df_fer67_IncV3.columns if (element.startswith('f_I') or element.startswith('f_F'))]
      merged_df_fer67_IncV3 = merged_df_fer67_IncV3[x]

      if merged_df_fer67_IncV3.isnull().sum().sum() > 0:
        print('problem with nan')
        merged_df_fer67_IncV3 = merged_df_fer67_IncV3.fillna(method = 'ffill')

      merged_df_fer67_IncV3 = merged_df_fer67_IncV3.values

      if merged_df_fer67_IncV3.shape[0]>= n_rows:
        data_to_predict = merged_df_fer67_IncV3[0:n_rows,:]
      else:
        data_to_predict = np.zeros((n_rows, merged_df_fer67_IncV3.shape[1]))
        dim_real = merged_df_fer67_IncV3.shape[0]
        reverse = merged_df_fer67_IncV3[::-1,:]
        data_to_predict[:dim_real,:] = merged_df_fer67_IncV3
        size_app = dim_real

        while  dim_real< n_rows:
          if n_rows> dim_real+size_app:
            data_to_predict[dim_real: (dim_real+size_app),:] = reverse
            reverse = reverse[::-1,:]
            dim_real = (dim_real+size_app)
          else:
            data_to_predict[dim_real:,:] = reverse[0: (n_rows - dim_real)]
            dim_real = n_rows

      p = classification_model.predict(data_to_predict[np.newaxis,:,:])

      print(f'          PREDICTION  {p} ')
      time_elapsed = time.time()-since
      data = {'video_name': nameVideo, 'prediction': p[0], 'time': time_elapsed}
      tot_prediction = pd.concat((tot_prediction, pd.DataFrame( data = data, index=[0])), ignore_index=True)
    else:
      videoNonCaricati = videoNonCaricati + 1
      time_elapsed = time.time()-since
      data = {'video_name': nameVideo, 'prediction': np.nan, 'time': time_elapsed}
      tot_prediction = pd.concat((tot_prediction, pd.DataFrame( data = data, index=[0])), ignore_index=True)

  except:
    videoNonCaricati = videoNonCaricati + 1
    time_elapsed = time.time()-since
    data = {'video_name': nameVideo, 'prediction': np.nan, 'time': time_elapsed}
    tot_prediction = pd.concat((tot_prediction, pd.DataFrame( data = data, index=[0])), ignore_index=True)

    print('[TIME: %.0f m %.0f s]'  %(time_elapsed // 60, time_elapsed % 60))

#tot_prediction.to_csv(dataMainDirectory + 'Results_prediction.csv', index=False)
print(tot_prediction)